In [158]:
#!pip install gensim pyLDAvis

In [154]:
import os
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import pymorphy2
import gensim
import stop_words
from nltk.stem.snowball import RussianStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import json
import string
import copy
import gensim
from gensim.models import Phrases
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel, CoherenceModel
import pyLDAvis.gensim_models

%matplotlib inline

In [35]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [14]:
news = []
with open('corpus.json', 'r', encoding="utf-8") as file:
    text = file.read()
    dct = json.loads(text)
data = pd.DataFrame(dct['catalog'])
data.head()

,article_id,category,title,text,tags
0,https://www.nn.ru/text/auto/2022/01/12/70369148/,auto,Кризис ОСАГО залатали рублем. Полисы резко под...,ОСАГО сильно прибавило для неблагополучных вод...,"[ОСАГО, автозакон, страховка]"
1,https://www.nn.ru/text/auto/2022/01/11/70366607/,auto,Я пропустил техосмотр в 2021 году. Оштрафуют л...,Обязательный техосмотр для частных машин отмен...,"[автозакон, техосмотр, автоликбез]"
2,https://www.nn.ru/text/auto/2022/01/08/70343681/,auto,Цены отмороженные: как кризис автомобильных пр...,Автомобили снова стали для большинства из нас ...,"[автопром, авторынок, цены на авто]"
3,https://www.nn.ru/text/auto/2022/01/07/70350704/,auto,"Новые Vesta и Logan, пятый Sportage и много «к...",Главная премьера начала года — новый KIA Sport...,"[автопром, авторынок, цены на авто]"
4,https://www.nn.ru/text/auto/2022/01/04/70343603/,auto,Бензин по 50 рублей — это лишь начало. Что жде...,Топливо дорожало весь прошлый год и установило...,"[АЗС, бензин, топливо, цены на топливо]"


In [15]:
data.text

0       ОСАГО сильно прибавило для неблагополучных вод...
1       Обязательный техосмотр для частных машин отмен...
2       Автомобили снова стали для большинства из нас ...
3       Главная премьера начала года — новый KIA Sport...
4       Топливо дорожало весь прошлый год и установило...
                              ...                        
4094    Поделиться   Предсказать массовые сокращения с...
4095    Поделиться   NN.RU: Трудоустройство инвалидов ...
4096    Поделиться   Эксперты МОТ обосновали предложен...
4097    commons.wikimedia.org   Поделиться   \- В Росс...
4098    Сбербанк   Поделиться   Сбербанк предлагает то...
Name: text, Length: 4099, dtype: object

In [16]:
data.category.unique()

array(['auto', 'gorod', 'health', 'job'], dtype=object)

In [18]:
labels = {'auto':0, 'gorod':1, 'health':2, 'job':3}

In [19]:
for cat in labels.keys():
    print(f'{data[data.category == cat].count()[0]} новостей в категории {cat}')

1036 новостей в категории auto
1024 новостей в категории gorod
1032 новостей в категории health
1007 новостей в категории job


In [111]:
corpus = []
categories = data.category.to_list()

for message in log_progress(data.text):
    corpus.append(message)

## Препроцессинг

In [112]:
corpus = np.asarray([doc.replace('Поделиться', '') for doc in corpus])
print(len(corpus))

4099


In [113]:
corpus = [re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', u'', doc) for doc in corpus]
corpus = [re.sub(' +' , ' ', doc) for doc in corpus]
corpus = np.asarray([doc.strip().strip('\t').replace('\n', u'')    for doc in corpus])

In [114]:
EXCLUDE_SYMBOLS_STR = u''.join(['№', '«', 'ђ', '°', '±', '‚', 'ћ', '‰', '…', '»', 'ѓ', 'µ', '·', 'ґ', 'њ', 'ї', 'џ', 'є', '‹',
                                '‡', '†', '¶', 'ќ', '€', '“', 'ў', '§', '„', '”', '\ufeff', '’', 'љ', '›', '•', '—', '‘', 
                                '\x7f', '\xad', '¤', '\xa0'])

GRAMMS = ['NOUN', 'ADJF', 'ADJS', 'PRTF', 'PRTS', 'GRND', 'ADVB']

regex_punct = re.compile('[%s]' % re.escape(string.punctuation))
regex_dig = re.compile('[%s]' % re.escape(string.digits))
regex_symb = re.compile('[%s]' % re.escape(EXCLUDE_SYMBOLS_STR))
regex_struct = re.compile('[%s]' % string.printable + string.whitespace)
emoji_pattern = re.compile("["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+")

In [115]:
corpus = [regex_punct.sub('', doc) for doc in corpus]
corpus = [regex_dig.sub('', doc) for doc in corpus]
corpus = [regex_symb.sub(' ', doc) for doc in corpus]
corpus = [regex_struct.sub('', doc) for doc in corpus]
corpus = [re.sub(' +' , ' ', doc.strip()) for doc in corpus]
corpus = [doc.lower() for doc in corpus]
corpus = [emoji_pattern.sub('', doc) for doc in corpus]
print(len(corpus))

4099


In [116]:
morph = pymorphy2.MorphAnalyzer()

corpus_tokens = []
inds_to_drop = []

for i, sentence in enumerate(log_progress(corpus[:])):
    tmp_tokens = []
    sp = sentence.split()
    for word in sp:
        if word not in stop_words.get_stop_words('ru'):
            if morph.word_is_known(word):
                tmp_tokens.append(word)
    if len(tmp_tokens) > 0:
        corpus_tokens.append(tmp_tokens)
    else:
        inds_to_drop.append(i)
        
print(len(corpus_tokens))

4095


In [117]:
inds_to_drop

[3281, 3308, 3433, 3529]

In [118]:
corpus = np.asarray(corpus)
labels = np.asarray(labels)

corpus = np.delete(corpus, inds_to_drop)
categories = np.delete(categories, inds_to_drop)

In [119]:
len(corpus), len(corpus_tokens), len(categories)

(4095, 4095, 4095)

In [120]:
stemmer = RussianStemmer()

corpus_tokens_stem = []

for i, tokens in enumerate(log_progress(corpus_tokens[:])):
    tmp = [stemmer.stem(word) for word in tokens]
    corpus_tokens_stem.append(tmp)
    
print(len(corpus_tokens_stem))

4095


In [121]:
tmp_corp = []

for i, tokens in enumerate(log_progress(corpus_tokens_stem[:])):
    tmp_corp.append([t for t in tokens if len(t) > 2])

result_corpus_tokens = tmp_corp

In [122]:
len(result_corpus_tokens)

4095

In [123]:
vocab = np.unique(np.concatenate(result_corpus_tokens).flatten()).tolist()
print(f'Количество токенов в словаре: {len(vocab)}')
print('Первые 10 слов в словаре:')
vocab[:10]

Количество токенов в словаре: 33729
Первые 10 слов в словаре:


['абака',
 'абакан',
 'абакум',
 'абакумов',
 'аббревиатур',
 'абвер',
 'абдоминальн',
 'абдулкерим',
 'аберрац',
 'абзац']

In [130]:
clean_text = copy.deepcopy(result_corpus_tokens)

In [131]:
bigram = Phrases(clean_text)
trigram = Phrases(bigram[clean_text])

for idx in range(len(clean_text)):
    for token in bigram[clean_text[idx]]:
        if '_' in token:
            clean_text[idx].append(token)
    for token in trigram[clean_text[idx]]:
        if '_' in token:
            clean_text[idx].append(token)

In [136]:
clean_text[1][-10:-1]

['административн_правонарушен',
 'част_стат_коап_наказыва',
 'езд_техосмотр',
 'уголовн_ответствен',
 'выезд_встречн',
 'превышен_скорост',
 'нов_коап',
 'штраф_езд',
 'оформлен_полис']

In [137]:
dictionary = Dictionary(clean_text)
dictionary.filter_extremes(no_below=10, no_above=0.1)

corpus1 = [dictionary.doc2bow(doc) for doc in clean_text]
print('Количество уникальных токенов: %d' % len(dictionary))
print('Количество документов: %d' % len(corpus1))

Количество уникальных токенов: 12085
Количество документов: 4095


In [143]:
chunksize = 2000
passes = 20
iterations = 400
eval_every = None

ldamodel = LdaModel(corpus=corpus1, num_topics=4, id2word=dictionary,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    passes=passes,
    eval_every=eval_every)

In [144]:
ldamodel.show_topics()

[(0,
  '0.007*"фот_арт" + 0.003*"модел" + 0.002*"миллион_рубл" + 0.002*"арт" + 0.002*"миллион" + 0.002*"верс" + 0.002*"дтп" + 0.002*"скорост" + 0.002*"транспортн_средств" + 0.002*"знак"'),
 (1,
  '0.010*"трудов" + 0.006*"договор" + 0.005*"трудов_договор" + 0.005*"трудов_кодекс" + 0.004*"размер" + 0.004*"выплат" + 0.004*"рабоч_мест" + 0.004*"российск_федерац" + 0.004*"штраф" + 0.003*"оплат_труд"'),
 (2,
  '0.005*"вакцин" + 0.003*"прививк" + 0.003*"вирус" + 0.003*"препарат" + 0.002*"лечен" + 0.002*"фот_александр" + 0.002*"фот_арт" + 0.002*"красн_зон" + 0.002*"антител" + 0.002*"организм"'),
 (3,
  '0.006*"рабоч_мест" + 0.005*"занят" + 0.005*"заработн_плат" + 0.004*"ваканс" + 0.004*"предприят" + 0.004*"центр_занят" + 0.004*"заработн" + 0.004*"тысяч_рубл" + 0.003*"служб_занят" + 0.003*"рынк_труд"')]

In [152]:
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim_models.prepare(ldamodel, corpus1, dictionary)

C:\Users\antid\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [157]:
data

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.174165 -0.011391       1        1  41.330596
0      0.165204  0.063594       2        1  39.020329
3     -0.117082 -0.191303       3        1  13.428632
1     -0.222287  0.139100       4        1   6.220443, topic_info=                           Term         Freq        Total Category  logprob  \
436                      трудов   878.000000   878.000000  Default  30.0000   
10635                    вакцин  1998.000000  1998.000000  Default  29.0000   
7125                 рабоч_мест  1061.000000  1061.000000  Default  28.0000   
2156                      занят   857.000000   857.000000  Default  27.0000   
453                     фот_арт  3295.000000  3295.000000  Default  26.0000   
...                         ...          ...          ...      ...      ...   
249                       штраф   215.135207   844.131891   Topic4  -5.6479   
11268               служб_занят   175.848253   635.705566   Topic4  -5.8495   
1692   гражданскоправов_договор   158.846106   272.057632   Topic4  -5.9512   
759                   так_образ   163.437231   710.704787   Topic4  -5.9227   
3846                   кандидат   162.893051   636.024357   Topic4  -5.9260   

       loglift  
436    30.0000  
10635  29.0000  
7125   28.0000  
2156   27.0000  
453    26.0000  
...        ...  
249     1.4103  
11268   1.4922  
1692    2.2393  
759     1.3075  
3846    1.4152  

[268 rows x 6 columns], token_table=       Topic      Freq                Term
term                                      
0          2  0.997885                авар
471        2  0.997882                 авт
472        2  0.996900             автоваз
803        2  0.999349             автомат
4          2  0.998325             автомоб
...      ...       ...                 ...
11470      3  0.995608  экономическ_активн
7846       1  0.995496             экскурс
7846       2  0.004945             экскурс
466        2  0.014252       юридическ_лиц
466        4  0.983362       юридическ_лиц

[421 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 2])

In [155]:
coherencemodel = CoherenceModel(model=ldamodel, texts=clean_text, dictionary=dictionary, coherence='c_v')

In [156]:
coherencemodel.get_coherence()

0.4284196396847615